In [1]:
import scda
import pprint
import logging
import os
import numpy as np
scda.configure_log()

# Prepare a design survey test to run on NCCS Discover

In [2]:
survey_params = {'Pupil': { 'prim': ['hex1', 'hex2', 'hex3', 'hex4'],
                            'secobs': 'X', 'thick': '025', 
                            'centobs': True, 'N': 125 },
                 'FPM': { 'rad': [3.0, 4.0, 5.0], 'M':60 },
                 'LS': { 'shape':'ann', 'obscure':0, 'aligntol':5, 'aligntolcon':3., 
                         'id':[20, 25, 30], 'od':[76, 78, 80, 82] },
                 'Image': { 'ida':-0.5, 'bw':0.10, 'Nlam':3}}

In [3]:
survey_dir = os.path.normpath("/astro/opticslab1/SCDA/Scripts/AMPL/april_survey01_10bw")
if not os.path.exists(survey_dir):
    os.mkdir(survey_dir)
    print("Created survey directory {:s}".format(survey_dir))
else:
    print("The survey directory {:s} already exists".format(survey_dir))

os.chdir(survey_dir)
survey_fname = os.path.basename(survey_dir)
ampl_src_dir = os.path.normpath("./amplsrc")
slurm_dir = os.path.normpath("./slurmsh")
sol_dir = os.path.normpath("./solutions")
log_dir = os.path.normpath("./logs")
TelAp_dir = os.path.normpath("../InputMasks/TelAp")
LS_dir = os.path.normpath("../InputMasks/LS")
FPM_dir = os.path.normpath("../InputMasks/FPM")

fileorg = {'work dir':'.', 'ampl src dir':ampl_src_dir, 'slurm dir':slurm_dir,
           'log dir':log_dir, 'sol dir':sol_dir,
           'TelAp dir':TelAp_dir, 'LS dir':LS_dir, 'FPM dir':FPM_dir}

Created survey directory /astro/opticslab1/SCDA/Scripts/AMPL/april_survey01_10bw


In [5]:
fileorg

{'FPM dir': '../InputMasks/FPM',
 'LS dir': '../InputMasks/LS',
 'TelAp dir': '../InputMasks/TelAp',
 'ampl src dir': 'amplsrc',
 'log dir': 'logs',
 'slurm dir': 'slurmsh',
 'sol dir': 'solutions',
 'work dir': '.'}

## Initiate a survey object with the above parameter combinations 

In [6]:
hexap_survey = scda.DesignParamSurvey(scda.QuarterplaneAPLC, survey_params, fileorg=fileorg)
print("This survey has {0:d} design parameter combinations.".format(hexap_survey.N_combos))
print("{0:d} parameters are varied: {1}".format(len(hexap_survey.varied_param_index), hexap_survey.varied_param_index))

This survey has 144 design parameter combinations.
4 parameters are varied: (('Pupil', 'prim'), ('FPM', 'rad'), ('LS', 'id'), ('LS', 'od'))


In [7]:
os.getcwd()

'/astro/opticslab1/SCDA/Scripts/AMPL/april_survey01_10bw'

## Show the file organization scheme

In [8]:
pprint.pprint(hexap_survey.fileorg)

{'FPM dir': '../InputMasks/FPM',
 'LS dir': '../InputMasks/LS',
 'TelAp dir': '../InputMasks/TelAp',
 'ampl src dir': 'amplsrc',
 'eval dir': '.',
 'log dir': 'logs',
 'slurm dir': 'slurmsh',
 'sol dir': 'solutions',
 'work dir': '.'}


## Show some attributes of an individual coronagraph object

In [9]:
#i = 100
i = 0
print("Telescope aperture file for design #{:d}: {:s}".format(i+1, hexap_survey.coron_list[i].fileorg['TelAp fname']))
print("Focal plane mask file for design #{:d}: {:s}".format(i+1, hexap_survey.coron_list[i].fileorg['FPM fname']))
print("Lyot stop file for design #{:d}: {:s}".format(i+1, hexap_survey.coron_list[i].fileorg['LS fname']))

Telescope aperture file for design #1: ../InputMasks/TelAp/TelAp_quart_hex1X025cobs1_N0125.dat
Focal plane mask file for design #1: ../InputMasks/FPM/FPM_quart_occspot_M060.dat
Lyot stop file for design #1: ../InputMasks/LS/LS_quart_ann20D76_clear_N0125.dat


### Check the status of input files needed to run the AMPL program

In [10]:
hexap_survey.check_ampl_input_files()

True

## List the varying parameter combinations

In [11]:
# pprint.pprint(hexap_survey.varied_param_combos)

## Write the batch of AMPL files

In [12]:
hexap_survey.write_ampl_batch(override_infile_status=False, overwrite=True)

INFO:root:Wrote all 144 of 144 design survey AMPL programs into amplsrc


## Write the batch of queue execution scripts

In [13]:
hexap_survey.write_slurm_batch(overwrite=True, queue_spec='12h')

INFO:root:Wrote all 144 of 144 design survey slurm scripts into slurmsh


## Write tables summarizing the design survey configuration and status to a spreadsheet

In [14]:
hexap_survey.write_spreadsheet()

INFO:root:Wrote design survey spreadsheet to ./april_survey01_10bw_ntz_2016-04-21.csv


## Store the design survey as a serialized python object

In [15]:
hexap_survey.write()

INFO:root:Wrote the design parameter survey object to ./april_survey01_10bw_ntz_2016-04-21.pkl


## Load an existing design survey

In [16]:
os.listdir('.')

['amplsrc',
 'solutions',
 'logs',
 'slurmsh',
 'april_survey01_10bw_ntz_2016-04-21.csv',
 'april_survey01_10bw_ntz_2016-04-21.pkl']

In [17]:
mysurvey = scda.load_design_param_survey(hexap_survey.fileorg['survey fname'])